# Mob session

In [46]:
import pandas as pd

df = pd.read_csv("influensa.csv", delimiter=";")
df.tail(10)

,Datum,Vårdform,Mått,DATA
2030,2025-03-27,Intensivvård,Antal,2.6
2031,2025-03-27,Intensivvård,Per 100 000,0.024
2032,2025-04-03,Slutenvård,Antal,.
2033,2025-04-03,Slutenvård,Per 100 000,.
2034,2025-04-03,Intensivvård,Antal,1.7
2035,2025-04-03,Intensivvård,Per 100 000,0.016
2036,2025-04-10,Slutenvård,Antal,.
2037,2025-04-10,Slutenvård,Per 100 000,.
2038,2025-04-10,Intensivvård,Antal,1.1
2039,2025-04-10,Intensivvård,Per 100 000,0.011


In [47]:
df.shape

(2040, 4)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2040 entries, 0 to 2039
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Datum     2040 non-null   object
 1   Vårdform  2040 non-null   object
 2   Mått      2040 non-null   object
 3   DATA      2040 non-null   object
dtypes: object(4)
memory usage: 63.9+ KB


In [49]:
import duckdb

df_antal = duckdb.query("""--sql
             SELECT *
             FROM df
            WHERE mått = 'Per 100 000'
""").df()
df_antal

,Datum,Vårdform,Mått,DATA
0,2015-07-09,Slutenvård,Per 100 000,0.000
1,2015-07-09,Intensivvård,Per 100 000,0.000
2,2015-07-16,Slutenvård,Per 100 000,0.000
3,2015-07-16,Intensivvård,Per 100 000,0.000
4,2015-07-23,Slutenvård,Per 100 000,0.008
...,...,...,...,...
1015,2025-03-27,Intensivvård,Per 100 000,0.024
1016,2025-04-03,Slutenvård,Per 100 000,.
1017,2025-04-03,Intensivvård,Per 100 000,0.016
1018,2025-04-10,Slutenvård,Per 100 000,.


In [50]:
df_100 = duckdb.query("""--sql
             SELECT *
             FROM df
            WHERE mått = ''
""").df()
df_100

,Datum,Vårdform,Mått,DATA


In [51]:
import numpy as np
#df['DATA'] = df['DATA'].str.replace(',', '.').astype(float)
df["DATA"] = df["DATA"].replace(r'^\.$', np.nan, regex=True).astype(float)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2040 entries, 0 to 2039
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Datum     2040 non-null   object 
 1   Vårdform  2040 non-null   object 
 2   Mått      2040 non-null   object 
 3   DATA      2036 non-null   float64
dtypes: float64(1), object(3)
memory usage: 63.9+ KB


In [77]:
df_antal = duckdb.query("""--sql
            SELECT *
            FROM df
            WHERE mått = 'Antal'
""").df()
df_antal

,Datum,Vårdform,Mått,DATA
0,2015-07-09,Slutenvård,Antal,0.0
1,2015-07-09,Intensivvård,Antal,0.0
2,2015-07-16,Slutenvård,Antal,0.0
3,2015-07-16,Intensivvård,Antal,0.0
4,2015-07-23,Slutenvård,Antal,0.6
...,...,...,...,...
1015,2025-03-27,Intensivvård,Antal,2.6
1016,2025-04-03,Slutenvård,Antal,NaN
1017,2025-04-03,Intensivvård,Antal,1.7
1018,2025-04-10,Slutenvård,Antal,NaN


In [59]:
df_antal['Datum'] = pd.to_datetime(df_antal['Datum'])


In [82]:
# Ensure Datum is a proper datetime object in the DataFrame
df_antal['Datum'] = pd.to_datetime(df_antal['Datum'])
 
# Now run the DuckDB query using EXTRACT
df_antal_grouped = duckdb.query("""--sql
    SELECT 
                                Vårdform,
        EXTRACT(YEAR FROM Datum) AS Year,
        SUM(CAST(DATA AS DOUBLE)) AS Total_Antal
    FROM df_antal
    WHERE Mått = 'Antal'
    GROUP BY Year, Vårdform
    ORDER BY Year
""").df()
df_antal_grouped

,Vårdform,Year,Total_Antal
0,Intensivvård,2015,6.0
1,Slutenvård,2015,56.1
2,Intensivvård,2016,51.0
3,Slutenvård,2016,632.7
4,Slutenvård,2017,617.9
5,Intensivvård,2017,30.3
6,Slutenvård,2018,1111.0
7,Intensivvård,2018,33.5
8,Slutenvård,2019,714.0
9,Intensivvård,2019,48.3


In [98]:
import plotly.express as px

fig = px.bar(df_antal_grouped, x="Year", y="Total_Antal", title= "Antal fall per år av <b>Intensivvård</b> och <b>Slutenvård</b>", color= "Vårdform", barmode="group")
#fig.update_layout(axix=dict(tickmode= "linear", dtick=1))
fig.update_layout(xaxis=dict(tickmode='linear',  # Viktigt för att tvinga visning av varje tick
                             dtick=1             # Visa varje år
                             ), 
                             yaxis= dict(title = "Antal"), 
                             plot_bgcolor="white"

                             )

fig.add_annotation(
    text=r"Åren med Covid-19🦠",
    yref='paper', xref='paper',
    x=0.55, y=0.4,  # Adjust y to position below the graph
    showarrow=False,
    font=dict(size=12, family="Times")
)

fig.show()
fig.write_html("mob_session.html")